In [1]:
from math import *
import numpy as np

def traverse(node,string=''):
    if isinstance(node, bin_tree):
        node=node.get_root()  
        # if unary
    if node.num_children == 1:
        string += (node.op)
        if node.coeff == None:
            string += ('('+traverse(node.left)+')')
        else:    
            string += ('('+str(node.coeff)+'*'+traverse(node.left)+')')
    # if binary
    elif node.num_children==2:
        string += '('+traverse(node.left)
        string += node.op
        string += traverse(node.right)+')'
    
    # if leaf node
    else:    
        if node.coeff == None:
            string += str(node.value)
        # if constant leaf node, will only have coefficient
        elif node.value == None:
            string += str(node.coeff)
        else:
            string += '('+str(node.coeff) + '*' + str(node.value)+')'
    return string

In [2]:
# intermediate_operands = ['sin', 'cos', 'exp', 'np.log', '+', '*']
# unary_operands = ['sin', 'cos', 'np.log', 'exp']
unary_operands = ['sin', 'cos', 'np.log']
binary_operands = ['*', '/', '+', '-', '**']
# terminal_operands = ["var1", "1"]
terminal_operands = ["c", "var1"]
coeff_max = 3



In [3]:
class PrintTree:
    """source: https://ebadahmadzadeh.wordpress.com/2014/02/26/how-to-print-a-dictionary-based-tree-in-python/"""
    def printTree(self, tree, d = 0):
        if (tree.num_children == 0):
            if tree.value is None:
                print("\t" * d, tree.index, tree.name, tree.coeff)
            else:
                if tree.coeff is not None:
                    print("\t" * d, tree.index, tree.name, tree.coeff + '*' + tree.value)
                else:
                    print("\t" * d, tree.index, tree.name, tree.value)
        else:
            if tree.coeff is None:
                print( "\t" * d, tree.index, tree.name, tree.op)
            else:
                print( "\t" * d, tree.index, tree.name, tree.op, tree.coeff)
            for i in [tree.left, tree.right]:
                if i != None:
                    self.printTree(i, d+1)
#                 else:
#                     print ("\t" * d, str('(') + "f" + str(')'))

In [4]:
for i in range(100):
    n1 = node(None, 1, "root")
    p = PrintTree()
    generate_tree(n1, [0.2, 0.4, 0.4], 0.08)
#     eval(traverse(n1))
    try:
        eval(traverse(n1))
    except:
        print(n1)
        print(traverse(n1))
        break
#     print(n1)
#     print(traverse(n1))
#     print(p.printTree(n1))
#     print(n1)

NameError: name 'node' is not defined

In [90]:
import copy
class bin_tree():
    def __init__(self, delta):
        self.root = node("root")
        self.delta = delta
        self.num_nodes = 0
        self.node_list = []
        self.coeff_list = []
        
    def generate_tree(self, node, c):
        """
        generates tree recursively from a starter node
        
        :param node: node object 
        
        :param c: array containing probabilities: [P(0 children), P(Unary), P(Binary)]
        
        :param delta: amount by which c parameters increase/decrease per each level of tree
        
        :return out: String containing either None or "constant" - used to check if subtree is 
        a root node with a constant
        """
        node.probs = c
        
        out = None # variable that tells whether node was constant or variable
        self.num_nodes += 1
        node.index = self.num_nodes
        self.node_list.append(node)
        r = np.random.rand()
        #decrease p(double branch) while
        #increase p(no branch)
        c = copy.deepcopy(c)
        c[0] += self.delta
        c[1] -= self.delta/2
        c[2] -= self.delta/2
        # no operands
        if r < c[0]:
#             node.coeff = np.random.uniform(coeff_max)
            val = np.random.choice(terminal_operands)
            # If a constant is chosen as a leaf node, then the c {index of node} is created
            if val == "c":    
                node.value = None
                node.coeff = val + str(node.index)
                out = "constant"
            else:
                node.value = val
        # unary operation
        elif r > c[0] and (r - c[0]) < c[1]:
            node.init_left()
            node.op = np.random.choice(unary_operands)
#             node.coeff = np.random.uniform(coeff_max)
            self.generate_tree(node.left, c)
        # binary
        else:
            node.init_left()
            node.init_right()
            node.op = np.random.choice(binary_operands)
            left = self.generate_tree(node.left, c) 
            right = self.generate_tree(node.right, c)
            # If the left and right children are constants, then convert current node into leaf node
            if left == "constant" and right == "constant":
                # erase children and assign evaluated value 
                node.value = None
                node.coeff = "c" + str(node.index)
                out = "constant"
                node.left=None
                node.right=None
                node.num_children = 0
                
            # if one branch is constant and operator is '*', store constant
            # as attribute of other branch, and replace current node with 
            # the child node. 
            elif (left == "constant" or right == "constant") and (node.op == "*" or node.op == "/"):
                if left == "constant":
                    node.right.coeff = 'c'+str(node.index)
                    node.right.parent = node.parent
                    # parent node points to child of current node
                    # so that child can replace current node
                    if node.name != "root":
                        if node.name == "left":
                            node.parent.left = node.right
                        else:
                            node.parent.right = node.right
                    node.right.name = node.name
                    node.right.index = node.index
                else:
                    node.left.coeff = 'c'+str(node.index)
                    node.left.parent = node.parent
                    if node.name != "root":
                        if node.name == "left":
                            node.parent.left = node.left
                        else:
                            node.parent.right = node.left
                    node.left.name = node.name
                    node.left.index = node.index
        
            # If operator is * or /, then combine the coefficients of both branches
            elif (left != "constant" and right != "constant") and (node.op == "*" or node.op == "/"):
                node.coeff = "c"+str(node.index)
                node.left.coeff = None
                node.right.coeff = None

                
        return out
            
    def get_root(self):
        return self.root 
            
    def __repr__(self):
        return self.traverse(self.get_root())
    
    def print_tree(self):
        pass
    
    def del_subtree(self, node):
        if node.name == "left":
            node.parent.left = None
        elif node.name == "right":
            node.parent.right = None
        self.reorder_whole_tree()
    
    def mutate_node(self, n):
        if isinstance(n, node):
            self.generate_tree(node, node.probs)
        elif isinstance(n, int):
            self.generate_tree(self.node_list[n], self.node_list[n].probs)
        self.reorder_whole_tree()

    def traverse(self, node=None, string=""):
        if node is None:
            node = self.get_root()
        if node.num_children == 1:
            string += (node.op)
            if node.coeff == None:
                string += ('('+self.traverse(node.left)+')')
            else:    
                string += ('('+str(node.coeff)+'*'+self.traverse(node.left)+')')
        # if binary
        elif node.num_children==2:
            string += '('+self.traverse(node.left)
            string += node.op
            string += self.traverse(node.right)+')'

        # if leaf node
        else:    
            if node.coeff == None:
                string += str(node.value)
            # if constant leaf node, will only have coefficient
            elif node.value == None:
                string += str(node.coeff)
            else:
                string += '('+str(node.coeff) + '*' + str(node.value)+')'
        return string
    
    def index_tree(self, node):
        """depth wise travels down tree and indexes each node. returns list of nodes in same order"""
        self.node_list.append(node)
        node.index = self.num_nodes
        self.num_nodes += 1
        for i in [node.left, node.right]:
            if i is not None:
                self.index_tree(i)
                
    def reorder_whole_tree(self):
        self.num_nodes = 0
        self.node_list = []
        self.index_tree(self.get_root())
    
    #list of coeffs
    # delete node function
    # maintain s ame coeffs for c when deleting node

In [91]:
class node():
    
#     def __init__(self, index, name, parent=None):
#         self.name = name
#         self.num_children=0
#         self.index=index
#         self._left=None
#         self._right=None
#         self._op=None
#         self.coeff=None
#         self._value = None
#         self.parent = parent
        
#     def init_left(self, index):
#         left_node = node(index, "left", parent=self)
#         self.left = left_node
#         self.num_children += 1
        
#     def init_right(self, index):
#         right_node = node(index, "right", parent=self)
#         self.right = right_node
#         self.num_children += 1 


    def __init__(self, name, parent=None):
        self.name = name
        self.num_children=0
        self.index=None
        self._left=None
        self._right=None
        self._op=None
        self.coeff=None
        self._value = None
        self.probs = None
        self.parent = parent
        
    def init_left(self):
        left_node = node("left", parent=self)
        self.left = left_node
        self.num_children += 1
        
    def init_right(self):
        right_node = node("right", parent=self)
        self.right = right_node
        self.num_children += 1 
    
    @property    
    def left(self):
        return self._left
    
    @property    
    def right(self):
        return self._right
    
    @property    
    def op(self):
        return self._op
    
    @property    
    def value(self):
        return self._value
    
    @value.setter
    def value(self, val):
        self._value = val
    
    @left.setter
    def left(self, node):
        self._left = node
    
    @right.setter
    def right(self, node):
        self._right = node

    @op.setter
    def op(self, op):
        self._op = op
    
    def __repr__(self):
        if self.num_children > 0:
            return "(left: {}, op: {}, right: {})".format(
                self.left, self.op, self.right)
        return "{}*{}".format(self.coeff, self.value)

In [92]:
a = bin_tree(delta=0.08)
c=[0.1, 0.2]
c.append(1-sum(c))
a.generate_tree(a.get_root(), c)
a.reorder_whole_tree()
p = PrintTree()
print(a.traverse())
# eval(a.traverse())


(((((var1+c7)-(var1*(c12+var1)))*(var1/var1))*sin(((var1+c24)-(var1+var1))))*(sin((var1-c32))/((var1/((var1-(c43*var1))-c46))**((c49-var1)+(((c58**(c59*var1))/var1)**var1)))))


In [93]:
p.printTree(a.get_root())

 0 root * c1
	 1 left *
		 2 left *
			 3 left -
				 4 left +
					 5 left var1
					 6 right c7
				 7 right * c8
					 8 left var1
					 9 right +
						 10 left c12
						 11 right var1
			 12 right /
				 13 left var1
				 14 right var1
		 15 right sin
			 16 left -
				 17 left +
					 18 left var1
					 19 right c24
				 20 right +
					 21 left var1
					 22 right var1
	 23 right /
		 24 left sin
			 25 left -
				 26 left var1
				 27 right c32
		 28 right **
			 29 left / c34
				 30 left var1
				 31 right -
					 32 left -
						 33 left var1
						 34 right c43*var1
					 35 right c46
			 36 right +
				 37 left -
					 38 left c49
					 39 right var1
				 40 right **
					 41 left / c56
						 42 left **
							 43 left c58
							 44 right c59*var1
						 45 right var1
					 46 right var1


In [94]:
a.mutate_node(20)

In [95]:
# a.generate_tree(a.node_list[19], [0.1, 0.1, 0.8])

In [96]:
p.printTree(a.get_root())

 0 root * c1
	 1 left *
		 2 left *
			 3 left -
				 4 left +
					 5 left var1
					 6 right c7
				 7 right * c8
					 8 left var1
					 9 right +
						 10 left c12
						 11 right var1
			 12 right /
				 13 left var1
				 14 right var1
		 15 right sin
			 16 left -
				 17 left +
					 18 left var1
					 19 right c24
				 20 right + c48
					 21 left var1
					 22 right var1
	 23 right /
		 24 left sin
			 25 left -
				 26 left var1
				 27 right c32
		 28 right **
			 29 left / c34
				 30 left var1
				 31 right -
					 32 left -
						 33 left var1
						 34 right c43*var1
					 35 right c46
			 36 right +
				 37 left -
					 38 left c49
					 39 right var1
				 40 right **
					 41 left / c56
						 42 left **
							 43 left c58
							 44 right c59*var1
						 45 right var1
					 46 right var1


In [97]:
#now to make the swap

a = bin_tree(delta=0.12)
c=[0.1, 0.2]
c.append(1-sum(c))
a.generate_tree(a.get_root(), c)
a.reorder_whole_tree()
p.printTree(a.get_root())

b = bin_tree(delta=0.12)
c=[0.1, 0.2]
c.append(1-sum(c))
b.generate_tree(b.get_root(), c)
b.reorder_whole_tree()
p.printTree(b.get_root())

 0 root /
	 1 root + c1
		 2 left + c3
			 3 left var1
			 4 right var1
		 5 right c8
	 6 right c9
 0 root **
	 1 left / c2
		 2 left +
			 3 left var1
			 4 right c5*var1
		 5 right -
			 6 left - c9
				 7 left var1
				 8 right c12
			 9 right **
				 10 left c15
				 11 right var1
	 12 right -
		 13 left sin
			 14 left c19
		 15 right c20


In [121]:
def swap_subtrees(tree1, tree2):
    """randomly selects two subtrees and swaps"""
    choice1 = np.random.choice(tree1.node_list)
    choice2 = np.random.choice(tree2.node_list)
    print("index of node from tree1: {}, repr: {}\n".format(choice1.index, choice1))
    print("index of node from tree2: {}, repr: {}\n".format(choice2.index, choice2))
    swap_parents(choice1, choice2)
    # re-index each node in the tree
    tree1.reorder_whole_tree()
    tree2.reorder_whole_tree()
    
    
def swap_parents(node1, node2):
    """remaps the parents of two randomly selected nodes"""
    parent_node1, left_node1, right_node1, val_node1, coeff_node1, op_node1, num_children_node1 = get_relatives(node1)
    parent_node2, left_node2, right_node2, val_node2, coeff_node2, op_node2, num_children_node2 = get_relatives(node2)
    
    node1.parent = parent_node2
    node1.coeff = coeff_node2
    node1.value = val_node2
    node1.left = left_node2
    node1.right = right_node2
    node1.op = op_node2
    node1.num_children = num_children_node2
    
    node2.parent = parent_node1
    node2.coeff = coeff_node1
    node2.value = val_node1
    node2.left = left_node1
    node2.right = right_node1
    node2.op = op_node1
    node2.num_children = num_children_node1
    
def get_relatives(node):
    return [node.parent, node.left, node.right, node.value, node.coeff, node.op, node.num_children]


In [122]:
# c = copy.deepcopy(a)
# d = copy.deepcopy(b)

In [132]:
a = bin_tree(delta=0.04)
c=[0.1, 0.2]
c.append(1-sum(c))
a.generate_tree(a.get_root(), c)
a.reorder_whole_tree()
p.printTree(a.get_root())

b = bin_tree(delta=0.10)
c=[0.1, 0.2]
c.append(1-sum(c))
b.generate_tree(b.get_root(), c)
b.reorder_whole_tree()
p.printTree(b.get_root())

 0 root * c1
	 1 left var1
	 2 right /
		 3 left **
			 4 left var1
			 5 right -
				 6 left var1
				 7 right c8
		 8 right +
			 9 left * c10
				 10 left +
					 11 left sin
						 12 left c13
					 13 right cos
						 14 left c15
				 15 right var1
			 16 right sin
				 17 left c18
 0 root -
	 1 left cos
		 2 left **
			 3 left -
				 4 left + c5
					 5 left - c7
						 6 left c10
						 7 right var1
					 8 right c12
				 9 right * c16
					 10 left -
						 11 left var1
						 12 right c21
					 13 right var1
			 14 right c23*var1
	 15 right -
		 16 left **
			 17 left var1
			 18 right +
				 19 left c30
				 20 right * c35
					 21 left +
						 22 left +
							 23 left var1
							 24 right c39
						 25 right c40
					 26 right var1
		 27 right / c42
			 28 left +
				 29 left var1
				 30 right -
					 31 left ** c46
						 32 left var1
						 33 right c49
					 34 right var1
			 35 right *
				 36 left var1
				 37 right +
					 38 left c57
					 39 right +
						 40 lef

In [133]:
#before
print("a:")
p.printTree(a.get_root())
print("b:")
p.printTree(b.get_root())


#after
swap_subtrees(a, b)
print("after:\n")
print("a:")
p.printTree(a.get_root())
print("b:")
p.printTree(b.get_root())

a:
 0 root * c1
	 1 left var1
	 2 right /
		 3 left **
			 4 left var1
			 5 right -
				 6 left var1
				 7 right c8
		 8 right +
			 9 left * c10
				 10 left +
					 11 left sin
						 12 left c13
					 13 right cos
						 14 left c15
				 15 right var1
			 16 right sin
				 17 left c18
b:
 0 root -
	 1 left cos
		 2 left **
			 3 left -
				 4 left + c5
					 5 left - c7
						 6 left c10
						 7 right var1
					 8 right c12
				 9 right * c16
					 10 left -
						 11 left var1
						 12 right c21
					 13 right var1
			 14 right c23*var1
	 15 right -
		 16 left **
			 17 left var1
			 18 right +
				 19 left c30
				 20 right * c35
					 21 left +
						 22 left +
							 23 left var1
							 24 right c39
						 25 right c40
					 26 right var1
		 27 right / c42
			 28 left +
				 29 left var1
				 30 right -
					 31 left ** c46
						 32 left var1
						 33 right c49
					 34 right var1
			 35 right *
				 36 left var1
				 37 right +
					 38 left c57
					 39 right +
						 

In [75]:
print(a)

((var1*(c2+(var1+(var1-c11))))/(var1-c15))


In [83]:
a.node_list[4].op

'/'

In [84]:
traverse(a.get_root())

'((var1*(c2+(var1+(var1-c11))))/(var1-c15))'